In [1]:
import transformers
import datasets
import torch

import peft

In [2]:
BASE_MODEL = 'cyberagent/open-calm-7b'
MODEL_DIR = '/scratch/acf15478nt/models/40812999/cyberagent/open-calm-7b/'

In [3]:
base_model = transformers.AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
peft_model = peft.PeftModel.from_pretrained(base_model, MODEL_DIR)

In [5]:
tokenizer = transformers.AutoTokenizer.from_pretrained(BASE_MODEL)

In [6]:
instruction = 'Cryptocurrency（暗号通貨）とは？'
input = '暗号通貨、暗号通貨、または暗号は、政府や銀行などの中央当局に支持や維持を依存しないコンピュータネットワークを通じて交換媒体として機能するように設計されたデジタル通貨です[2]。取引の当事者が持っていると主張するお金を持っていることを検証する分散型システムであり、2つのエンティティ間で資金移動する際に銀行などの従来の仲介者を不要にします[3]。'

# instruction = "Linkedinが行っていることをまとめてください。"
# input = "LinkedIn（/lɪŋ）は、ウェブサイトとモバイルアプリで動作するビジネスと雇用に特化したソーシャルメディアプラットフォームである。2003年5月5日に開始され、現在はマイクロソフトの所有となっている。このプラットフォームは主に人脈作りとキャリア形成に使用され、求職者は履歴書を、雇用主は求人を掲載することができる。2015年から同社の収益のほとんどは、リクルーターや営業担当者にメンバーの情報へのアクセスを販売することで得られたものである。2016年12月から、マイクロソフトの完全子会社となった。2023年3月現在、LinkedInは200以上の国や地域から9億人以上の登録会員を有している。 LinkedInでは、会員（労働者と雇用者の両方）がプロフィールを作成し、現実の職業上の関係を表すオンラインのソーシャルネットワークで互いにつながることができます。メンバーは、誰でも（既存のメンバーであるかどうかにかかわらず）コネクションになるよう招待することができる。LinkedInは、オフラインイベントの開催、グループへの参加、記事の執筆、求人情報の公開、写真やビデオの投稿などにも使用できる。"

# instruction = "以下の文を3行でまとめてください"
# input = "【イスタンブール時事】パレスチナ自治区ガザ情勢の緊張緩和に向けた国際会議「カイロ平和サミット」が21日、エジプトのカイロで行われた。会議では各国からガザで軍事作戦を続けるイスラエルへの厳しい非難の声が相次ぎ、パレスチナ自治政府のアッバス議長はイスラエルが民間人に対する「無差別攻撃を行っている」と訴えた。イスラエルは7日に同国内への奇襲攻撃を仕掛けたイスラム組織ハマスの壊滅を目指すが、アッバス氏は「住民の避難所になっている病院や学校も標的になっている」と主張。ガザへの人道支援も「イスラエルが妨害してきた」と指摘した。ヨルダンのアブドラ国王は、イスラエルによる「戦争犯罪」を糾弾した。南アフリカのラマポーザ大統領らもイスラエルの対パレスチナ政策を批判した。会議を呼び掛けたエジプトのシシ大統領はガザで深刻化する人道状況を憂慮し、「危機解決に向けたロードマップ」策定への意欲を示した。サウジアラビアのファイサル外相は、イスラエルによるガザでの住民退避通告の関連で、「パレスチナ人の強制移住の試みを全て拒否する」と述べた。国連のグテレス事務総長は、ガザ住民に支援物資を届けるための「人道停戦」を提唱した。会議にはアラブ首長国連邦（UAE）のムハンマド大統領や欧州連合（EU）のミシェル大統領らも参加。日本からは上川陽子外相が出席した。ロイター通信によると、ガザで軍事作戦を続けるイスラエルに強い影響力を持つ米国は、駐エジプト臨時代理大使の出席にとどめた。"

prompt=f"""### Instruction:
{instruction}
### Input:
{input}
### Response:
"""

In [7]:
# ベースモデルでの文章生成
with peft_model.disable_adapter():
    with torch.no_grad():
        tokens = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False, add_special_tokens=False).to(peft_model.device)
        outputs = peft_model.generate(**tokens, max_new_tokens=1024)

    print("-------- ベースモデルでの文章生成結果 ----------")
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


-------- ベースモデルでの文章生成結果 ----------
### Instruction:
Cryptocurrency(暗号通貨)とは?
### Input:
暗号通貨、暗号通貨、または暗号は、政府や銀行などの中央当局に支持や維持を依存しないコンピュータネットワークを通じて交換媒体として機能するように設計されたデジタル通貨です[2]。取引の当事者が持っていると主張するお金を持っていることを検証する分散型システムであり、2つのエンティティ間で資金移動する際に銀行などの従来の仲介者を不要にします[3]。
### Response:
暗号通貨は、暗号化技術を使用して、電子的に保存されるお金です。それは、電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存されるお金である、または電子的に保存さ

In [8]:
with torch.no_grad():
    tokens = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False, add_special_tokens=False).to(peft_model.device)
    outputs = peft_model.generate(**tokens, max_new_tokens=1024)

print("-------- ファインチューニング済みのモデルでの文章生成結果 ----------")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


-------- ファインチューニング済みのモデルでの文章生成結果 ----------
### Instruction:
Cryptocurrency(暗号通貨)とは?
### Input:
暗号通貨、暗号通貨、または暗号は、政府や銀行などの中央当局に支持や維持を依存しないコンピュータネットワークを通じて交換媒体として機能するように設計されたデジタル通貨です[2]。取引の当事者が持っていると主張するお金を持っていることを検証する分散型システムであり、2つのエンティティ間で資金移動する際に銀行などの従来の仲介者を不要にします[3]。
### Response:
暗号通貨は、政府や銀行などの従来の仲介者なしに、2つのエンティティ間で資金移動する際に銀行などの従来の仲介者を不要にするデジタル通貨です。
